## Imports and stuff

In [31]:
import tensorflow as tf
import keras
from keras.layers import Dense,Dropout, Activation
from keras.models import Sequential
from keras.optimizers import *

import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import librosa
from sklearn.linear_model import LogisticRegression, Lasso, ElasticNet
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import scale, PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore")

In [32]:
X_train = np.array(np.load("X_train.npy"))
y_train = np.array(np.load("y_train.npy"))
X_dev = np.array(np.load("X_dev.npy"))
y_dev = np.array(np.load("y_dev.npy"))
X_test = np.array(np.load("X_test.npy"))
y_trainonehot = np.array(pd.get_dummies(y_train))
y_devonehot = np.array(pd.get_dummies(y_dev))
print(X_train.shape, X_test.shape, X_dev.shape)

(582, 193) (298, 193) (290, 193)


## Neural Network

In [33]:
y_trainonehot = np.array(pd.get_dummies(y_train))
y_devonehot = np.array(pd.get_dummies(y_dev))
X_tot = np.concatenate([X_train, X_dev, X_test])
X_tot = scale(X_tot)
"""X_tot = PolynomialFeatures(interaction_only=True).fit_transform(X_tot)
lol = PCA(n_components = 500)
X_tot = lol.fit_transform(X_tot)"""
X_train, X_dev, X_test = X_tot[:len(X_train)], X_tot[len(X_train):len(X_train)+len(X_dev)], X_tot[len(X_train)+len(X_dev):]
print(X_train.shape, y_train.shape, X_dev.shape, y_dev.shape, X_test.shape)

(582, 193) (582,) (290, 193) (290,) (298, 193)


In [34]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [36]:
scores = []
cv = 1
for i in range(cv):
    model = Sequential()
    model.add(Dense(40, input_dim = 193))
    model.add(Dropout(0.1))
    model.add(Activation('relu'))
    model.add(Dense(20))
    model.add(Dropout(0.1))
    model.add(Activation('relu'))
    model.add(Dense(15))
    model.add(Activation('softmax'))
    #optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-4)
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    X = np.concatenate([X_train, X_dev])
    y = np.concatenate([y_trainonehot, y_devonehot])
    X, y = unison_shuffled_copies(X, y)
    
    history = model.fit(X, y, batch_size = 10, nb_epoch = 40, 
                        verbose = 0, validation_split = 0.1)
    score1 = model.evaluate(X_train,y_trainonehot,verbose=0)
    score2 = model.evaluate(X_dev,y_devonehot,verbose=0)
    print(score1[1], score2[1])
    scores.append((score1[1]+score2[1])/2)
print("Mean score:", np.mean(scores))

0.994845360825 0.996551724138
Mean score: 0.995698542481


In [37]:
from sklearn.metrics import confusion_matrix
L = [np.argmax(x) for x in model.predict(X_dev)]
print(confusion_matrix(L,y_dev))

[[21  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 20  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 19  0  0  0  0  0  0  1  0  0  0  0  0]
 [ 0  0  0 19  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 19  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 18  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 21  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 18  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 18  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 17  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 23  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 18  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 21  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0 19  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0 18]]


In [14]:
%matplotlib inline
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.ylim((0,1))
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [38]:
nn_pred = model.predict(X_test)
nn_pred = [np.argmax(nn_pred[i]) for i in range(len(nn_pred))]
np.savetxt('y_pred.txt', nn_pred, fmt="%d")
print(np.unique(nn_pred,return_counts=True))

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14], dtype=int64), array([16, 25, 16, 13, 20, 24, 33, 16, 25, 23,  9, 13, 29, 14, 22], dtype=int64))


## Regression Logistique ftw

In [39]:
reg = LogisticRegression()
reg.fit(np.concatenate([X_train,X_dev]),np.concatenate([y_train,y_dev]))
#clf.fit(X_dev, y_dev)
reg_pred = reg.predict(X_test)
np.savetxt('y_pred.txt', reg_pred, fmt="%d")

## SVM

In [40]:
from sklearn.svm import SVC
X = np.concatenate([X_train,X_dev])
y = np.concatenate([y_train,y_dev])
X,y = unison_shuffled_copies(X, y)
svm = SVC()
svm.fit(X,y)
svm_pred = svm.predict(X_test)
np.savetxt('y_pred.txt', svm_pred, fmt="%d")
print(np.unique(svm_pred,return_counts=True))
print(svm.score(X,y))

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]), array([15, 27, 15,  9, 20, 19, 30, 21, 32, 23, 10, 11, 21, 26, 19], dtype=int64))
0.979357798165


# K-means

In [41]:
from sklearn.cluster import KMeans
import operator
dict_list = []
for i in range(15):
    dict_list.append({})
kmeans = KMeans(n_clusters=15, random_state=0).fit(X_tot)
X = np.concatenate([X_train,X_dev])
y = np.concatenate([y_train,y_dev])
pred = kmeans.predict(X)
#print(pred, y)
for i in range(len(pred)):
    try:
        dict_list[pred[i]][y[i]] += 1
    except:
        dict_list[pred[i]][y[i]] = 1
#print(dict_list)
corr_dict = {}
for i in range(15):
    try:
        corr_dict[i] = max(dict_list[i].items(), key=operator.itemgetter(1))[0]
    except:
        corr_dict[i] = None
        
for i in range(len(pred)):
    pred[i] = corr_dict[pred[i]]

In [42]:
y_new = []
pred_new = []
new_class = {0:1, 1:1, 3:1, 4:1, 13:1, 14:1, 9:1,    5:5, 11:5, 12:5,    2:7, 6:7, 7:7, 8:7, 10:7}
for i in range(len(y)):
    try:
        y_new.append(new_class[y[i]])        
    except:
        y_new.append(y[i])
    try:
        pred_new.append(new_class[pred[i]])
    except:
        pred_new.append(pred[i])
from sklearn.metrics import confusion_matrix, accuracy_score
print(confusion_matrix(y_new,pred_new))
print(accuracy_score(y_new,pred_new))

[[244  85  74]
 [  7 150  20]
 [ 52  26 214]]
0.697247706422


## XGBoost 

In [43]:
import os
mingw_path = 'C:\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']
import xgboost as xgb

gbm = xgb.XGBClassifier()
X = np.concatenate([X_train,X_dev])
y = np.concatenate([y_train,y_dev])
X,y = unison_shuffled_copies(X, y)
    
print(X.shape,y.shape)
gbm.fit(X, y)

(872, 193) (872,)


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [44]:
xgboost_pred = gbm.predict(X_test)
np.savetxt('y_pred.txt', xgboost_pred, fmt="%d")
print(np.unique(xgboost_pred,return_counts=True))

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]), array([12, 19, 12, 22, 17, 13, 25, 24, 21, 26, 16, 24, 30,  8, 29], dtype=int64))


## Mix results 

In [45]:
new_pred = []
from collections import Counter
preds = {}
for i in range(len(nn_pred)):
    preds[i] = []
    preds[i].append(nn_pred[i])
    preds[i].append(reg_pred[i])
    preds[i].append(xgboost_pred[i])
    preds[i].append(svm_pred[i])
for x in preds.values():
    count = Counter(x)
    new_pred.append(count.most_common()[0][0])
np.savetxt('y_pred.txt', new_pred, fmt="%d")
print(np.unique(new_pred,return_counts=True))

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14], dtype=int64), array([17, 26, 19, 12, 20, 21, 28, 16, 28, 25, 10, 14, 27, 11, 24], dtype=int64))


## Test

In [61]:
from sklearn.neighbors import KNeighborsClassifier 
clf = KNeighborsClassifier(n_neighbors = 3)
clf.fit(X,y)
y = clf.predict(X_test)
np.savetxt('y_pred.txt', y, fmt="%d")